In [105]:
# import sys; sys.path.append('common/');
from helper import *
import sys, importlib
importlib.reload(sys.modules['helper'])
from helper import *

DATA_PATH = '/projects/persuasionforgood-master/Face_acts/dialogue_act_prediction/resisting-persuasion/data'

df = pd.read_csv(DATA_PATH+'/neg_final_convs.csv')
display(df.head())

df = df.rename(columns={"Resistance Labels": "fine_labels"})

valid_ids = []
all_ids = list(set(list(df['Index'])))

spam_text ='''C_2bdb308591df45e29a321a7cfd167002
C_a7e41f175b6f4a53ab31daa0bb02de8b
C_91995f19b3664f4dad37c6b738be5bf4
C_e93957571b1140e48dffc52829252ed7
C_d26bad533e364c4d9b8358d777ad3205
C_399c1f48c33b4561aa7ce7652e953682
C_7e57a726bcb045df8e2105da21341f9a
C_c6a45e6e9d624babbf8b5cdc008f7f36
C_9e3c0dea33da49a8bde97a19e62f3bf7
C_6b1dae69c75941d39d021a4c38d1e3df
C_1f50c4606e234f8f956bdf6356300ad6
C_dc99d0923cf74332a0e4eb8f832da0a9
C_13ac299de52c434a88565e0ec3f9f387
C_78ef20d98c2143dfa452136cda053592
C_38bfc703b8e04023b6379861fd1ae55f
C_efe36c78d44c43d6947efab9a72c979e'''

spam_ids = set(spam_text.split('\n'))
print(len(spam_ids))

for id_ in all_ids:
    temp_df = df[df['Index']==id_]
    for index, row in temp_df.iterrows():
        label = row['fine_labels']
        if label ==label:
            valid_ids.append(id_)
            break

valid_ids = list(set(valid_ids)-spam_ids)
print(len(df), len(all_ids), len(set(valid_ids)))
df = df.loc[(df['Index'].isin(valid_ids))]


,Unnamed: 0,Index,User,Text,Resistance Labels,Unnamed: 5,Unnamed: 6
0,33,C_e2781ecd69264b00a4cc25ee96d271c5,1,Hello,NaN,NaN,NaN
1,34,C_e2781ecd69264b00a4cc25ee96d271c5,0,Hi there. I had a few questions about the phon...,NaN,NaN,NaN
2,35,C_e2781ecd69264b00a4cc25ee96d271c5,1,Sure,NaN,NaN,NaN
3,36,C_e2781ecd69264b00a4cc25ee96d271c5,0,How old is the phone? I don`t see it stated an...,Source Derogation-Contesting,NaN,NaN
4,37,C_e2781ecd69264b00a4cc25ee96d271c5,1,2 years old,NaN,NaN,NaN


16
12373 879 703


In [106]:
df['fine_labels'] = df['fine_labels'].fillna('not-a-strategy')
df['resistance_labels'] = df['fine_labels']

with open(DATA_PATH+'/neg.pkl','rb') as handle:
    neg_pickle = pickle.load(handle)
    

conv_dict = {}
for conv in neg_pickle['train']:
    conv_dict[conv['uuid']] = conv
for conv in neg_pickle['test']:
    conv_dict[conv['uuid']] = conv
for conv in neg_pickle['valid']:
    conv_dict[conv['uuid']] = conv

all_uuids = list([conv for conv in conv_dict])
valid_ids = list(set(all_uuids)&set(valid_ids))
random.shuffle(valid_ids)

train_ids_dict = {}
test_ids_dict  = {}

random.shuffle(all_uuids)
for count in range(0,5):
    test_ids_dict[count]  = [valid_id for i, valid_id in enumerate(valid_ids) if i%5==count] 
    train_ids_dict[count] = [i for i in valid_ids if i not in test_ids_dict[count]]

In [107]:
print(set(df['resistance_labels']))

{'not-a-strategy', 'Self Pity-Empowement', 'Counter Argumentation-Contesting', 'Hesitance-Hesitance', 'Self Assertion-Empowerment', 'Personal Choice-Empowerment', 'Source Derogation-Contesting', 'Information Inquiry-Contesting'}


## create data jsons for the Hi-GRU model

In [108]:
affect_file= '../data/Resources/NRC_affect.txt'
affect_df= pd.read_csv(affect_file, sep='\t', lineterminator='\n')
display(affect_df.head())

print(set(affect_df['AffectDimension']))

vad_file= '../data/Resources/NRC_vad.txt'
vad_df= pd.read_csv(vad_file, sep='\t', lineterminator='\n')

display(vad_df.head())

emotion_file= '../data/Resources/NRC_emotion.txt'
emo_df= pd.read_csv(emotion_file, sep='\t', lineterminator='\n')
display(emo_df.head())
print(set(emo_df['Emotion']))

with open('../data/Resources/word_categories.p','rb') as handle:
    word_categories_dict= pickle.load(handle)

with open('../data/Resources/starred_word_categories.p','rb')as handle:
    starred_word_categories_dict= pickle.load(handle)

liwc_categories=['Funct','Pronoun','Ppron','I','We','You','SheHe','They','Ipron','Article','Verbs','AuxVb','Past','Present','Future','Adverbs','Prep','Conj','Negate','Quant','Numbers','Swear','','Family','Friends','Humans','Affect','Posemo','Negemo','Anx','Anger','Sad','CogMech','Insight','Cause','Discrep','Tentat','Certain','Inhib','Incl','Excl','Percept','See','Hear','Feel','Body','Health','Sexual','Ingest','Relativ','Motion','Space','Time','Work','Achiev','Leisure','Home','Money','Relig','Death','Assent','Nonflu','Filler','Other']	

print(len(liwc_categories))

def create_vad_dict(vad_df):
    vad_dict={}
    for index, row in vad_df.iterrows():
        w= row['Word']
        vad_dict[w]={}
        vad_dict[w]['Valence']=row['Valence']
        vad_dict[w]['Arousal']= row['Arousal']
        vad_dict[w]['Dominance']= row['Dominance']
    
    return vad_dict

def create_emo_dict(emo_df):
    emo_dict={}
    emotions_set=list({'trust', 'fear', 'surprise', 'sadness', 'positive', 'joy', 'anger', 'negative', 'disgust', 'anticipation'})
    for index, row in emo_df.iterrows():
        w= row['Word']
        if w not in emo_dict:
            emo_dict[w]={}
            for emotion in emotions_set:
                emo_dict[w][emotion]=0
        emo_dict[w][row['Emotion']]= row['Presence']
    
    return emo_dict

def create_affect_dict(affect_df):
    affect_dict={}
    affect_set= list({'joy', 'sadness', 'fear', 'anger'})
    for index, row in affect_df.iterrows():
        w= row['term']
        if w not in affect_dict:
            affect_dict[w]={}
            for affect in affect_set:
                affect_dict[w][affect]=0
        affect_dict[w][row['AffectDimension']]= row['score']
    return affect_dict

vad_dict= create_vad_dict(vad_df)
emo_dict= create_emo_dict(emo_df)
affect_dict= create_affect_dict(affect_df)


import numpy as np

def create_embeddings(word_list, vads, affects, emos):
    
    vad_sent_dict={}
    affect_sent_dict={}
    emo_sent_dict={}
    
    for elem in vads:
        vad_sent_dict[elem]=[]
    for elem in affects:
        affect_sent_dict[elem]=[]
    for elem in emos:
        emo_sent_dict[elem]=[]
    
    for w in word_list:
        if w in vad_dict:
            for elem in vads:
                try:
                    vad_sent_dict[elem].append(float(vad_dict[w][elem]))
                except Exception as e:
                    vad_sent_dict[elem].append(0)
        else:
            for elem in vads:
                vad_sent_dict[elem].append(0)
        
        if w in affect_dict:
            for elem in affects:
                try:
                    affect_sent_dict[elem].append(float(affect_dict[w][elem]))
                except Exception as e:
                    affect_sent_dict[elem].append(0)
        else:
            for elem in affects:
                affect_sent_dict[elem].append(0)
        
        if w in emo_dict:
            for elem in emos:
                try:
                    emo_sent_dict[elem].append(float(emo_dict[w][elem]))
                except Exception as e:
                    emo_sent_dict[elem].append(0)
        else:
            for elem in emos:
                emo_sent_dict[elem].append(0)
            
    vad_features    = []
    affect_features = []
    emo_features    = []
    
    for elem in vads:
        val = np.mean(vad_sent_dict[elem])
        if val== val:
            vad_features.append(val)
        else:
            vad_features.append(0)
            
    for elem in affects:
        val = np.mean(affect_sent_dict[elem])
        if val== val:
            affect_features.append(val)
        else:
            affect_features.append(0)
    for elem in emos:
        val = np.mean(emo_sent_dict[elem])
        if val== val:
            emo_features.append(val)
        else:
            emo_features.append(0)
        
    return vad_features, affect_features, emo_features

def create_liwc(words):
    category_tag={}
    count=0
    for category in liwc_categories:
        category_tag[category]=0
    category_tag['Other']=0	

    for word in words:
        flag1=0
        flag2=0

        if word_categories_dict[word]!=[]:
            flag1=1

        for category in word_categories_dict[word]:
            if category not in category_tag:
                category_tag[category]=1
            else:
                category_tag[category]+=1

        for substring in starred_word_categories_dict.keys():
            if word.startswith(substring):
                flag2=1
                for category in starred_word_categories_dict[substring]:
                    if category not in category_tag:
                        category_tag[category]=1
                    else:
                        category_tag[category]+=1

        if flag1==0 and flag2==0:
            category_tag['Other']+=1
        count+=1

    if count>0:
        for i in category_tag:
            category_tag[i]=category_tag[i]/count

    return category_tag

vads=['Valence', 'Arousal', 'Dominance']
affects=sorted(list({'joy', 'sadness', 'fear', 'anger'}))
emos=sorted(list({'trust', 'fear', 'surprise', 'sadness', 'positive', 'joy', 'anger', 'negative', 'disgust', 'anticipation'}))
liwc_categories=['Funct','Pronoun','Ppron','I','We','You','SheHe','They','Ipron','Article','Verbs','AuxVb','Past','Present','Future','Adverbs','Prep','Conj','Negate','Quant','Numbers','Swear','','Family','Friends','Humans','Affect','Posemo','Negemo','Anx','Anger','Sad','CogMech','Insight','Cause','Discrep','Tentat','Certain','Inhib','Incl','Excl','Percept','See','Hear','Feel','Body','Health','Sexual','Ingest','Relativ','Motion','Space','Time','Work','Achiev','Leisure','Home','Money','Relig','Death','Assent','Nonflu','Filler','Other']	

,term,score,AffectDimension
0,outraged,0.964,anger
1,brutality,0.959,anger
2,hatred,0.953,anger
3,hateful,0.940,anger
4,terrorize,0.939,anger


{'fear', 'anger', 'sadness', 'joy'}


,Word,Valence,Arousal,Dominance
0,aaaaaaah,0.479,0.606,0.291
1,aaaah,0.520,0.636,0.282
2,aardvark,0.427,0.490,0.437
3,aback,0.385,0.407,0.288
4,abacus,0.510,0.276,0.485


,Word,Emotion,Presence
0,aback,anger,0
1,aback,anticipation,0
2,aback,disgust,0
3,aback,fear,0
4,aback,joy,0


{'positive', 'anticipation', 'sadness', 'surprise', 'negative', 'trust', 'fear', 'anger', 'disgust', 'joy'}
64


In [54]:
! pip install vaderSentiment

     |████████████████████████████████| 125 kB 3.2 MB/s eta 0:00:01


In [118]:
print(len(conv_dict))

ratios = sorted([conv_dict[conv]['ratio'] for conv in conv_dict])
ratio_bar = np.median(ratios)



5956


In [119]:
import re
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

only_words = r'\b\w+\b'

exception_arr = []

def normalize_arr(arr):
    if np.sum(arr)>0:
        return arr/np.sum(arr)
    else:
        return np.zeros(len(arr))
    
def sentiment_analyze(text):
    analyzer = SentimentIntensityAnalyzer()
    vs = analyzer.polarity_scores(text)
    return [vs['pos'],vs['neg'], vs['neu']]


def create_conv(temp_df, train_flag = True):
    try:
        curr_id      = list(temp_df['Index'])[0]
    except Exception as e:
        import pdb; pdb.set_trace()
    

    convs      = []
    conv       = []
    max_utt_len  = 0
    print(temp_df.columns)
    print(len(temp_df), len(set(temp_df['Index'])))

    max_utt_len = 0 
    row_cnt     = 0
    
    for index, row in temp_df.iterrows():
        row_cnt+=1
        print('Done for {}'.format(row_cnt), end='\r')
        if row['Index']!= curr_id:
            for elem in conv:
                elem['bert-feat'].extend([0 for i in range(max_utt_len-len(elem['bert-feat']))])
            if conv != []:
                convs.append(conv)
            conv = []
            max_utt_len =0
            curr_id = row['Index']
            
        utt_dict = {}
        utt_dict['utterance'] = row['Text']
        utt = normalizeString(row['Text'])
        toks = tokenizer.convert_tokens_to_ids(tokenizer.tokenize('[CLS]')) +  tokenizer.convert_tokens_to_ids(tokenizer.tokenize(utt)) + tokenizer.convert_tokens_to_ids(tokenizer.tokenize('[SEP]'))
        max_utt_len = max(max_utt_len, len(toks))
        utt_dict['bert-feat'] = toks
        utt_dict['fine_labels'] = row['fine_labels']
        if row['fine_labels'] =='not-a-strategy':
            utt_dict['coarse_labels']= row['fine_labels']
        else:
            utt_dict['coarse_labels']= row['fine_labels'].split('-')[1]

        utt_dict['resistance_labels']= row['resistance_labels']
        utt_dict['speaker']= row['User']

        word_list= re.findall(only_words, row['Text'].lower().strip())
        vad_feats, affect_feats, emo_feats = create_embeddings(word_list, vads, affects, emos)

        for elem in vad_feats:
            if elem!=elem:
                import pdb; pdb.set_trace()

        liwc_feats= create_liwc(word_list)

        utt_dict['vad_features'] = vad_feats
        utt_dict['affect_features'] = affect_feats
        utt_dict['emo_features'] = emo_feats
        utt_dict['liwc_features'] = [liwc_feats[i] for i in liwc_categories]

        utt_dict['sentiments'] = sentiment_analyze(utt)
        
        if conv_dict[row['Index']]['ratio'] > ratio_bar:
            utt_dict['ratio_bucket']  = 1 
        else:
            utt_dict['ratio_bucket']  = 0

        conv.append(utt_dict)

    for elem in conv:
        elem['bert-feat'].extend([0 for i in range(max_utt_len-len(elem['bert-feat']))])

    if conv != []:
        convs.append(conv)

    print(len(convs))
    
    return convs

In [120]:
import json

for count in range(0,5):
    print('count {}'.format(count), end ='\r')
    train_ids = train_ids_dict[count]
    test_ids  = test_ids_dict[count]
    
    train= df.loc[df['Index'].isin(train_ids)]
    conv_json = create_conv(train, True)
    with open(DATA_PATH+'/higru_bert_data/train'+str(count)+'neg'+'.json','w') as handle:
        json.dump(conv_json,handle, indent=4)
    
    test= df.loc[df['Index'].isin(test_ids)]
    conv_json = create_conv(test, False)
    with open(DATA_PATH+'/higru_bert_data/test'+str(count)+'neg'+'.json','w') as handle:
        json.dump(conv_json,handle, indent=4)

    train_sentences = []
    test_sentences  = []

    for index, row in train.iterrows():
        train_sentences.append(row['Text'].lower().strip())

    for index, row in test.iterrows():
        test_sentences.append(row['Text'].lower().strip())

    print(len(test))
    print(len(train))

    cnt=0
    for elem in test_sentences:
        if elem in train_sentences:
            cnt+=1

    print(cnt/len(test_sentences))

Index(['Unnamed: 0', 'Index', 'User', 'Text', 'fine_labels', 'Unnamed: 5',
       'Unnamed: 6', 'resistance_labels'],
      dtype='object')
7582 562
633e for 7582
Index(['Unnamed: 0', 'Index', 'User', 'Text', 'fine_labels', 'Unnamed: 5',
       'Unnamed: 6', 'resistance_labels'],
      dtype='object')
1969 141
167e for 1969
1969
7582
0.07719654647028949
Index(['Unnamed: 0', 'Index', 'User', 'Text', 'fine_labels', 'Unnamed: 5',
       'Unnamed: 6', 'resistance_labels'],
      dtype='object')
7532 562
636e for 7532
Index(['Unnamed: 0', 'Index', 'User', 'Text', 'fine_labels', 'Unnamed: 5',
       'Unnamed: 6', 'resistance_labels'],
      dtype='object')
2019 141
164e for 2019
2019
7532
0.0633977216443784
Index(['Unnamed: 0', 'Index', 'User', 'Text', 'fine_labels', 'Unnamed: 5',
       'Unnamed: 6', 'resistance_labels'],
      dtype='object')
7656 562
642e for 7656
Index(['Unnamed: 0', 'Index', 'User', 'Text', 'fine_labels', 'Unnamed: 5',
       'Unnamed: 6', 'resistance_labels'],
      dt

In [123]:
set([utt['ratio_bucket'] for conv in conv_json for utt in conv])

{0, 1}

In [96]:
train_sentences = []
test_sentences  = []

for index, row in train.iterrows():
    train_sentences.append(row['Text'].lower().strip())

for index, row in test.iterrows():
    test_sentences.append(row['Text'].lower().strip())

print(len(test))
print(len(train))

count=0
for elem in test_sentences:
    if elem in train_sentences:
        count+=1

print(count/len(test_sentences))

1513
6287
0.8288169200264376


In [14]:
for count in range(0,5):
    print('count {}'.format(count), end ='\r')
    train_ids = train_ids_dict[count]
    test_ids  = test_ids_dict[count]
    
    train= df.loc[df['B2'].isin(train_ids)]
    train_labels = set(train['fine_labels'])
    
    test= df.loc[df['B2'].isin(test_ids)]
    test_labels = set(test['fine_labels'])
    
    print(len(train_labels), len(test_labels), len(test_labels - train_labels))
    

10 10 0
10 10 0
10 10 0
10 10 0
10 10 0


In [15]:
feature_dim_dict = {'vad_features': 3, 'affect_features': 4, 'emo_features': 10, 'liwc_features': 64, 'sentiments': 3, 'face_acts': 8, 'norm_er_strategies': 10, 'norm_er_DAs': 17, 'ee_DAs': 23, 'all': 3+4+10+64+3+8+10+17+23}

feature_dim_arr  = [i for i in feature_dim_dict if i!='all']
print(feature_dim_arr)

['vad_features', 'affect_features', 'emo_features', 'liwc_features', 'sentiments', 'face_acts', 'norm_er_strategies', 'norm_er_DAs', 'ee_DAs']
